In [ ]:
import polars as pl

## Basic Functions

Let's explore the main data-frame level operations:

- `select()`
- `filter()`
- `with_columns`
- `group_by` + `agg`
- `concat()`
- `union()`

We'll start with a toy dataset because this is a great way to play and debug easily.

In [ ]:
df = pl.DataFrame({
    'a': [1,1,1,2,2,2],
    'b': [1,2,3,4,5,6],
    'c': [7,8,9,0,1,2]
})

In [ ]:
df.select('a','b')

In [ ]:
df.filter( pl.col('c') > 5)

In [ ]:
df.with_columns(
    x = pl.col('a') + 7,
    y = pl.col('a') * pl.col('b'), 
    z = pl.col('c').min()
)

In [ ]:
df.group_by('a').agg( pl.col('b').sum() )

In [ ]:
df.group_by('a').agg( 
    pl.col('b').sum(),
    pl.col('b').mean().alias('b_avg') 
    )

In [ ]:
pl.concat([df, df])

In [ ]:
df2 = pl.DataFrame({'a': [1,2], 'x': ['a==1','a==2']})
df.join(df2, how = "left", on = 'a')

## Method Chaining

In [ ]:
(
    df
    .filter( pl.col('c') != 9)
    .with_columns( a = pl.col('a') + 1 )
    .group_by('a')
    .agg( pl.col('b').sum() )
)

## Aggregation

In [ ]:
df.select( pl.col('b').sum() )

In [ ]:
df.group_by('a').agg( pl.col('b').sum() )

In [ ]:
df.with_columns(pl.col('b').sum().over('a').alias('b_sum'))

## Case Study: Total EV by County

Using GA early vote data, count the number of early votes by county. 

What are the key steps?

1. Read in the data
2. Inspect the data
3. Group the data by county
4. Count the votes

In [ ]:
df_ev = pl.read_csv("GA_Early_Vote - GA_Early_Vote.csv")
df_ev.glimpse() 
# you can also use df_ev.head() but I find glimpse() better for 'wide' data

In [ ]:
(
    df_ev
    .group_by('vb_vf_county_name')
    .agg( pl.col('voterbase_id').count() )
    .head(5)
)

## Derived Fields

Within `with_columns()`, you can create one or more unique variables.

In [ ]:
(
    df
    .with_columns(
        b_low_high = pl.when( pl.col('b') < 4 )
                       .then( pl.lit('low') )
                       .otherwise( pl.lit('high') ),
        c_recode   = pl.col('c').replace({'0': None})        
    )
)

There are many useful helpful functions for this such as:

- Conditional Logic:
  + pl.when().then().otherwise()
  + pl.replace({'from':'to'})

- [String Manipulation](https://docs.pola.rs/api/python/stable/reference/expressions/string.html):
  + str.slice() 
  + str.split()
  + str.to_date() / str.striptime()
  + str.upper() / str.lower()

- Date Manipulation:
  + date() to construct dates
  + dt.year() / month() / day() to extract date parts
  + dt.strftime() to change date formatting

In [ ]:
df2 = pl.DataFrame({'a': ['apple','banana','carrot'], 'b': ['20200101','20200102','20200103']})
df2.with_columns(
    a_abbr = pl.col('a').str.slice(0,3),
    b_date = pl.col('b').str.to_date('%Y%m%d')
)

If you're using variables you just created, you'll need to do it in a separate call to `with_columns()`.

In [ ]:
# doesn't work
# df.with_columns(x = pl.col('a'), y = pl.col('x') + 1)

# works
(
df
  .with_columns(x = pl.col('a'))
  .with_columns(y = pl.col('x') + 1)
)

## Case Study: Top EV County in July

What 3 counties had the highest number of votes by or before July 31?

In [ ]:
df_ev.glimpse()

Let's breakdown the steps of what we need to do:

- Filter the data to only the date range of interest
  + Convert `august_primary_voted_date` to a sortable YYYY-MM-DD date field
  + Subset results to those with a value less than 2024-07-22
- Summarize the number of voters per day by each county
  + Which type of aggregation? We want **one row per county**
- Sort by the voter count, starting with the largest
- Subset results to the top 3

In [ ]:
(
    df_ev
    .filter(
        pl.col('aug_primary_voted_date').str.to_date('%m/%d/%Y') 
          <= pl.date(2024,7,31)
        )
    .group_by('vb_vf_county_name')
    .agg( pl.col('voterbase_id').n_unique().alias('n') )
    .sort('n', descending = True)
    .head(3)
)

## Window Functions

In [ ]:
df.with_columns(
    c_max_by_a = pl.col('c').max().over('a'),
    b_cum_by_a = pl.col('b').cum_sum().over('a'),
    b_cum_by_a_sort_c = pl.col('b').sort_by('c', descending=True).cum_sum().over('a')
)

## Column Name Selectors

In [ ]:
df4 = pl.DataFrame({
    'voter_id': [123,234,345,456],
    'dt_register': ['2024-01-01','2023-05-07','2023-12-12','2021-01-07'],
    'dt_dob': ['1953-01-01','1968-05-07','1993-12-12','2000-01-07'],
})

In [ ]:
import polars.selectors as cs

df4.with_columns(
  cs.starts_with('dt').str.to_date()
)

In [ ]:
df4.with_columns(
  cs.string().str.to_date()
)

In [ ]:
df4.with_columns(
  cs.starts_with('dt').str.to_date() ,
  cs.starts_with('dt').str.to_date().dt.year().name.prefix('year_')
)

## API Response Wrangling

In [ ]:
example_api_response = [
    {
        'name': 'Voter 1',
        'dob': '1990-10-25',
        'voter_info': {
            'precinct': 123,
            'state_hd': 2,
            'state_sen': 10,
            'us_hd': 40
        }
    },
    {
        'name': 'Voter 2',
        'dob': '1953-02-10',
        'voter_info': {
            'precinct': 456,
            'state_hd': 7,
            'state_sen': 11,
            'us_hd': 42
        }
    }
]

example_api_response

In [ ]:
pl.DataFrame(example_api_response)

In [ ]:
pl.DataFrame(example_api_response).unnest('voter_info')

## Case Study: Vote Counting Accumulation Rate

For all counties, calculate the cumulative percent of their early vote obtained each day, so that day n's value is:

`prop = (sum votes from day 1 to day n) / (total votes)`

Then, show the day-by-day results for Fulton County and create a list of the counties ordered by the date they hit 50%.

What do we need to do?

- Create one table with the cumulative proportion to use in the subsequent queries
  + Calculate votes by day by county
  + Calculate cumulative votes over the days within each county 
  + Calculate ultimate total votes for each county
  + Use the above two fields to calculate the cumulative proportion

- Use the table to show results for Fulton county
  + Filter to Fulton county
  + Sort results by day

- Use the table to list counties by the day they crossed 50%
  + Filter to where the proportion is >50%
  + Group by county
  + Find the minimum date
  + Order results by the minimum date

In [ ]:
df_cumul = (
    df_ev
    .with_columns( pl.col('aug_primary_voted_date').str.to_date('%m/%d/%Y') )
    .group_by('aug_primary_voted_date', 'vb_vf_county_name')
    .agg( n = pl.col('voterbase_id').n_unique() )
    .with_columns(
        cum_n = pl.col('n').cum_sum().over(
                                partition_by = 'vb_vf_county_name',
                                order_by = 'aug_primary_voted_date'),
        tot_n = pl.col('n').sum().over('vb_vf_county_name')
    )
    .with_columns(
        prop = pl.col('cum_n') / pl.col('tot_n')
    )

)

In [ ]:
(
  df_cumul
  .filter(pl.col('vb_vf_county_name') == pl.lit('FULTON'))
  .sort('aug_primary_voted_date')
)

In [ ]:
(
  df_cumul
  .filter( pl.col('prop') >= 0.5 )
  .group_by('vb_vf_county_name')
  .agg( pl.col('aug_primary_voted_date').min() )
  .sort('aug_primary_voted_date')
)